*All packages and methods found running as on 28Mar2024*

In [1]:
import sys
sys.executable

'/home/mansoor/anaconda3/envs/mobile_qna/bin/python'

In [ ]:
!ls 

In [ ]:
# !mkdir manuals

In [2]:
!ls manuals

OnePlus_5_User_Manual.pdf		   Samsung_Galaxy_S6_active_G890A.pdf
sam-f946-f731-en-um-os13-072723-final.pdf


In [ ]:
!pip install langchain
!pip install pinecone-client
!pip install pypdf
!pip install openai
!pip install tiktoken

In [3]:
from langchain.document_loaders import PyPDFDirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import OpenAIEmbeddings
from langchain.llms import OpenAI
from langchain.vectorstores import Pinecone
from langchain.chains import RetrievalQA
from langchain.prompts import PromptTemplate
import os

In [4]:
loader=PyPDFDirectoryLoader("manuals")

In [5]:
data=loader.load()

In [6]:
data[0]

Document(page_content='SMARTPHONE\nUser Manual\nPlease read this manual before operating your device \nand keep it for future reference.', metadata={'source': 'manuals/Samsung_Galaxy_S6_active_G890A.pdf', 'page': 0})

In [7]:
len(data)

328

In [8]:
data[327]

Document(page_content='', metadata={'source': 'manuals/OnePlus_5_User_Manual.pdf', 'page': 47})

In [9]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=20)

In [10]:
text_chunks = text_splitter.split_documents(data)

In [11]:
text_chunks

[Document(page_content='SMARTPHONE\nUser Manual\nPlease read this manual before operating your device \nand keep it for future reference.', metadata={'source': 'manuals/Samsung_Galaxy_S6_active_G890A.pdf', 'page': 0}),
 Document(page_content='i\nLegal NoticesWarning: This product contains chemicals known \nto the State of California to cause cancer and reproductive toxicity. For more information, please call 1-800-SAMSUNG (726-7864).\nNote: Water-resistant and dustproof based on IP68 \nrating, which tests submersion up to 1.5 meters for up \nto 30 minutes.\nIntellectual Property', metadata={'source': 'manuals/Samsung_Galaxy_S6_active_G890A.pdf', 'page': 1}),
 Document(page_content='All Intellectual Property, as defined below, owned by or which is otherwise the property of Samsung or its respective suppliers relating to the SAMSUNG Phone, including but not limited to, accessories, parts, or software relating there to (the “Phone System”), is proprietary to Samsung and protected under fe

In [12]:
len(text_chunks)

854

In [13]:
print(text_chunks[0].page_content)

SMARTPHONE
User Manual
Please read this manual before operating your device 
and keep it for future reference.


In [14]:
print(text_chunks[1].page_content)

i
Legal NoticesWarning: This product contains chemicals known 
to the State of California to cause cancer and reproductive toxicity. For more information, please call 1-800-SAMSUNG (726-7864).
Note: Water-resistant and dustproof based on IP68 
rating, which tests submersion up to 1.5 meters for up 
to 30 minutes.
Intellectual Property


In [15]:
print(text_chunks[850].page_content)

• Avoid using dirty sockets when charging your phone. Doing so may cause your phone to fail to charge properly .
• Do not remove the SIM card while the phone is transmitting data or saving information.
• Avoid using modified accessories, which may damage the device or violate local regulations for radio devices.
• If applicable, the user should turn off 5GHz Wi-Fi in outdoor environments.


In [ ]:
!pip install python-dotenv

In [16]:
from dotenv import load_dotenv
load_dotenv()

True

In [17]:
embedding=OpenAIEmbeddings()

/home/mansoor/anaconda3/envs/mobile_qna/lib/python3.9/site-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The class `langchain_community.embeddings.openai.OpenAIEmbeddings` was deprecated in langchain-community 0.0.9 and will be removed in 0.2.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import OpenAIEmbeddings`.
  warn_deprecated(


In [ ]:
# embedding

In [18]:
embed_length = len(embedding.embed_query("hi i am fine"))
embed_length

1536

## Create Embeddings for each of the Text Chunk

In [ ]:
!pip install -U langchain-pinecone

In [19]:
# pinecone UI doc to create index

from pinecone import Pinecone, PodSpec

pc = Pinecone(api_key = os.getenv('PINECONE_API_KEY'))
index_name = 'mobile-qna'

pc.create_index(
  name=index_name,
  dimension=embed_length,
  metric="cosine",
  spec=PodSpec(
    environment="gcp-starter"
  )
)


https://python.langchain.com/docs/integrations/vectorstores/pinecone

In [20]:
from langchain_pinecone import PineconeVectorStore
    
docsearch = PineconeVectorStore.from_documents(text_chunks, embedding, index_name=index_name)

In [21]:
docsearch

In [22]:
docsearch.as_retriever()

VectorStoreRetriever(tags=['PineconeVectorStore', 'OpenAIEmbeddings'], vectorstore=<langchain_pinecone.vectorstores.PineconeVectorStore object at 0x7f7612172310>)

In [24]:
query= "Can you tell me how I can configure a wall paper"

In [25]:
docs = docsearch.similarity_search(query)

In [26]:
docs

[Document(page_content='based on the time and weather of your current location. (Lock screen only.)Wallpaper Settings', metadata={'page': 87.0, 'source': 'manuals/Samsung_Galaxy_S6_active_G890A.pdf'}),
 Document(page_content='82 Wallpaper Settings\nSettingsChange the look of the home \nand lock screens with wallpaper. \nYou can display a favorite picture \nor choose from preloaded wallpapers.\n1. From a Home screen, tap  Apps > \xa0Settings  \n> PERSONAL tab > Wallpaper.\n2. Tap Home screen, Lock screen, or Home and \nlock screen.\n3. Tap a image to preview it, and then tap \nSET\xa0AS\xa0WALLPAPER.\n• Tap From Gallery for additional wallpapers.\n• Tap Live weather to add high quality images', metadata={'page': 87.0, 'source': 'manuals/Samsung_Galaxy_S6_active_G890A.pdf'}),
 Document(page_content='To access Wallpaper settings:\n ►From a Home screen, tap \n Apps > \xa0Settings  \n> PERSONAL tab > Wallpaper.\nWidgets\nWidgets are self-contained apps that run on a Home\xa0screen. There ar

In [27]:
for cnt, doc in enumerate(docs, 1):
    print(f'{cnt} --> {doc}')
    

1 --> page_content='based on the time and weather of your current location. (Lock screen only.)Wallpaper Settings' metadata={'page': 87.0, 'source': 'manuals/Samsung_Galaxy_S6_active_G890A.pdf'}
2 --> page_content='82 Wallpaper Settings\nSettingsChange the look of the home \nand lock screens with wallpaper. \nYou can display a favorite picture \nor choose from preloaded wallpapers.\n1. From a Home screen, tap  Apps > \xa0Settings  \n> PERSONAL tab > Wallpaper.\n2. Tap Home screen, Lock screen, or Home and \nlock screen.\n3. Tap a image to preview it, and then tap \nSET\xa0AS\xa0WALLPAPER.\n• Tap From Gallery for additional wallpapers.\n• Tap Live weather to add high quality images' metadata={'page': 87.0, 'source': 'manuals/Samsung_Galaxy_S6_active_G890A.pdf'}
3 --> page_content='To access Wallpaper settings:\n ►From a Home screen, tap \n Apps > \xa0Settings  \n> PERSONAL tab > Wallpaper.\nWidgets\nWidgets are self-contained apps that run on a Home\xa0screen. There are all kinds of wid

In [ ]:
!pip install langchain_openai

In [28]:
from langchain_openai import OpenAI
llm = OpenAI()
# llm

In [29]:
qa = RetrievalQA.from_chain_type(llm=llm, chain_type="stuff", retriever=docsearch.as_retriever())

In [30]:
query = "Can you tell me how I can configure a wall paper"

In [ ]:
!pwd

In [31]:
qa.invoke(query)

{'query': 'Can you tell me how I can configure a wall paper',
 'result': ' To configure a wallpaper, first go to your Home screen and either pinch the screen or touch and hold an empty space. Then, tap on "Wallpapers" and select the desired wallpaper from the options provided. You can also access the Wallpaper Settings by going to Apps > Settings > PERSONAL tab > Wallpaper. From there, you can choose to set the wallpaper for your Home screen, Lock screen, or both.'}

In [32]:
def get_my_response(query):
    resp = qa.invoke(query)
    print(resp.get('result'))

In [33]:
response = qa.invoke('how to do powersharing between disconnected devices')
response

{'query': 'how to do powersharing between disconnected devices',
 'result': ' To do powersharing between disconnected devices, go to Settings > Battery and device care > Battery > Wireless powersharing and turn it on. Then, with the phone facedown, place the compatible device on the back of the phone to charge. Keep in mind that some features may not be available while sharing power and the location of the wireless charging coil may vary by device, so you may need to adjust the placement to make a connection. Also, make sure to remove any accessories or covers for best results.'}

In [34]:
get_my_response('how to do powersharing between disconnected devices')

 To do powersharing between disconnected devices, follow these steps:

1. Go to Settings and tap on Battery and device care.

2. Tap on Battery and then Wireless powersharing.

3. Tap on Battery limit and choose a percentage. Once the device you are charging reaches this level of charge, wireless powersharing will automatically turn off.

4. Tap on the toggle to turn on the charging feature.

5. With the phone facedown, place the compatible device on the back of the phone to charge it.

Keep in mind that powersharing may not work properly if you are using a certain type of accessory or cover. Additionally, the location of the wireless charging coil may vary by device, so you may need to adjust the placement to make a connection. A notification or vibration will occur when charging begins, indicating that you have made a connection. If you have trouble connecting or charging is slow, remove any cover from each device. It is also recommended to remove any accessories or cover before usin

In [35]:
import sys
while True:
  user_input = input(f"Input Prompt: ")
  if user_input == 'exit':
    print('Exiting')
    sys.exit(f'terminating the bot...')
  if user_input == '':
    continue
  get_my_response(user_input)

Input Prompt:  Can you tell me about Oxygen OS


 OxygenOS is a fast, clean and customizable operating system based on Android. It is constantly refined and updated based on user feedback, offering a smooth and flexible user experience.


Input Prompt:  Can you tell me about Samsung Galaxy S6


 I'm sorry, but I don't know about the specifics of the Samsung Galaxy S6 as it is not mentioned in the given context. However, you can visit the Samsung support website for more information on your device.


Input Prompt:  What is flex mode


...

 I don't know, as it is subjective and depends on personal preferences and needs. It is recommended to compare the specifications and features of both phones to determine which one suits your needs better.


Input Prompt:  exit


Exiting


SystemExit: terminating the bot...

/home/mansoor/anaconda3/envs/mobile_qna/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3534: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [36]:
pc.delete_index(index_name)

In [37]:
!rm requirements.txt

In [38]:
!pip freeze > requirements.txt

In [39]:
!cat requirements.txt | wc -l

138
